## Memoria Temporal: Buffer Memory

* Most of memory-related functionality in LangChain is marked as beta
    1. Most functionality are not production ready
    2. Most functionality wirk with Legacy chains instead of LCEL
* The main exception to this is the ChatMessageHistory functionality

In [1]:
import os
from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv())
openai_api_key = os.environ["OPENAI_API_KEY"]

In [2]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model="gpt-3.5-turbo-0125")

buffer refers to a temporary storage area in memory used to hold data.

In [4]:
from langchain_core.prompts import (
    ChatPromptTemplate,
    MessagesPlaceholder,
    SystemMessagePromptTemplate,
    HumanMessagePromptTemplate
)
from langchain.chains import LLMChain
from langchain.memory import ConversationBufferMemory

In [5]:
prompt = ChatPromptTemplate(
    messages=[
        SystemMessagePromptTemplate.from_template(
            "You are a nice chatbot having a conversation with a human."
        ),
        #The variable_name here is what must align with memory
        MessagesPlaceholder(variable_name="chat_history"),
        HumanMessagePromptTemplate.from_template("{question}")
    ]
)

# Notice that we 'return_messages=True' to fit into MessagesPlaceholder
# Notice that 'chat_history' aligns with the MessagePlaceholder name.
memory = ConversationBufferMemory(memory_key="chat_history", return_messages=True)

conversation = LLMChain(
    llm=llm,
    prompt=prompt,
    verbose=True,
    memory=memory
)

C:\Users\jarai\AppData\Local\Temp\ipykernel_23264\1117788240.py:16: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 1.0. Use RunnableSequence, e.g., `prompt | llm` instead.
  conversation = LLMChain(


In [8]:
conversation.invoke({'question': 'hi'})



> Entering new LLMChain chain...
Prompt after formatting:
System: You are a nice chatbot having a conversation with a human.
Human: hi
AI: Hello! How are you today?
Human: hi

> Finished chain.


{'question': 'hi',
 'chat_history': [HumanMessage(content='hi'),
  AIMessage(content='Hello! How are you today?'),
  HumanMessage(content='hi'),
  AIMessage(content="Hello again! What's on your mind today?")],
 'text': "Hello again! What's on your mind today?"}

In [9]:
conversation({'question': 'My name is Javier and I am 33 years old'})

C:\Users\jarai\AppData\Local\Temp\ipykernel_23264\993019606.py:1: LangChainDeprecationWarning: The method `Chain.__call__` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use invoke instead.
  conversation({'question': 'My name is Javier and I am 33 years old'})




> Entering new LLMChain chain...
Prompt after formatting:
System: You are a nice chatbot having a conversation with a human.
Human: hi
AI: Hello! How are you today?
Human: hi
AI: Hello again! What's on your mind today?
Human: My name is Javier and I am 33 years old

> Finished chain.


{'question': 'My name is Javier and I am 33 years old',
 'chat_history': [HumanMessage(content='hi'),
  AIMessage(content='Hello! How are you today?'),
  HumanMessage(content='hi'),
  AIMessage(content="Hello again! What's on your mind today?"),
  HumanMessage(content='My name is Javier and I am 33 years old'),
  AIMessage(content="Nice to meet you, Javier! It's great to know a bit about you. Is there anything specific you'd like to talk about or share today?")],
 'text': "Nice to meet you, Javier! It's great to know a bit about you. Is there anything specific you'd like to talk about or share today?"}

In [13]:
print(memory.buffer)

[HumanMessage(content='hi'), AIMessage(content='Hello! How are you today?'), HumanMessage(content='hi'), AIMessage(content="Hello again! What's on your mind today?"), HumanMessage(content='My name is Javier and I am 33 years old'), AIMessage(content="Nice to meet you, Javier! It's great to know a bit about you. Is there anything specific you'd like to talk about or share today?")]


you can limit the number of conversational exchanges stored in memory. In example below we are limiting the stored memory to 3 questions

In [10]:
from langchain.memory import ConversationBufferWindowMemory
from langchain.chains import ConversationChain

In [11]:
window_memory = ConversationBufferWindowMemory(k=3)

conversation_window = ConversationChain(
    llm=llm,
    memory = window_memory,
    verbose=True
)

C:\Users\jarai\AppData\Local\Temp\ipykernel_23264\1320835759.py:3: LangChainDeprecationWarning: The class `ConversationChain` was deprecated in LangChain 0.2.7 and will be removed in 1.0. Use RunnableWithMessageHistory: https://python.langchain.com/v0.2/api_reference/core/runnables/langchain_core.runnables.history.RunnableWithMessageHistory.html instead.
  conversation_window = ConversationChain(


## Conversation Token Buffer Memory

It is similar to the prevous one but it limits with tokens instead of questions

In [14]:
from langchain.memory import ConversationTokenBufferMemory

In [15]:
token_memory = ConversationTokenBufferMemory(
    llm=llm,
    max_token_limit=50
)

## Conversation Summary Memory

Sotres a summary of the previous conversation exchanges

In [16]:
from langchain.memory import ConversationSummaryBufferMemory

This is useful as it less costly but is less precise at the same time

In [17]:
summary_memory = ConversationSummaryBufferMemory(
    llm=llm, 
    max_token_limit=100
)

# Memoria Permanente : Chat message history

In [18]:
from langchain.memory import ChatMessageHistory

history = ChatMessageHistory()

history.add_user_message('hi')
history.add_ai_message('whats up?')

In [19]:
history.messages

[HumanMessage(content='hi'), AIMessage(content='whats up?')]